In [1]:
import pathlib
from importlib import reload
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray as cfxr

In [2]:
ds = xr.open_dataset("tests/data/SPI1_ERA5_China_sample0.nc")
ds

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 660, lon: 43, lat: 26)
Coordinates:
  * time     (time) datetime64[ns] 5kB 1960-01-31 1960-02-29 ... 2014-12-31
  * lon      (lon) float64 344B 72.0 73.5 75.0 76.5 ... 130.5 132.0 133.5 135.0
  * lat      (lat) float64 208B 17.25 18.75 20.25 21.75 ... 51.75 53.25 54.75
Data variables:
    SPI1     (time, lat, lon) float32 3MB ...
Attributes:
    CDI:          Climate Data Interface version 2.4.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Dec 25 22:30:49 2024: cdo -b F32 -selyear,1960/2014 -se...
    CDO:          Climate Data Operators version 2.4.0 (https://mpimet.mpg.de...

# Stack locations

In [3]:
import dominosee.utils
from dominosee.utils import stack_lonlat
reload(dominosee.utils)

<module 'dominosee.utils' from '/home/hmwang-cli/PycharmProjects/dominosee-dev/dominosee/utils.py'>

In [4]:
da = stack_lonlat(ds.SPI1)
da

<xarray.DataArray 'SPI1' (time: 660, location: 1118)> Size: 3MB
array([[-0.41666552, -0.33389136, -0.50240225, ..., -1.4563528 ,
        -0.85959387, -0.6144137 ],
       [-0.2137151 , -0.29338124, -0.35434958, ...,  1.2927388 ,
         0.5341902 , -0.16195127],
       [-0.13538474, -0.11242806, -0.7112601 , ..., -0.76348215,
        -0.29436743, -0.01121015],
       ...,
       [ 1.0034107 ,  0.6416878 ,  0.21670395, ...,  0.3627314 ,
        -0.109614  , -0.71738636],
       [ 0.32788515,  0.6170307 ,  0.2810465 , ..., -0.00572755,
        -0.25397933, -0.01543918],
       [ 0.79709935,  1.1792182 ,  1.1986301 , ...,  0.7904675 ,
         1.1455151 ,  1.119474  ]], dtype=float32)
Coordinates:
  * time      (time) datetime64[ns] 5kB 1960-01-31 1960-02-29 ... 2014-12-31
  * location  (location) object 9kB MultiIndex
  * lat       (location) float64 9kB 17.25 17.25 17.25 ... 54.75 54.75 54.75
  * lon       (location) float64 9kB 72.0 73.5 75.0 76.5 ... 132.0 133.5 135.0
Attributes:
    calibration_period:  
    freq:                ME
    window:              1
    scipy_dist:          gamma
    method:              APP
    group:               time.month
    time_indexer:

# Extract event layers

In [5]:
import dominosee.eventorize
reload(dominosee.eventorize)
from dominosee.eventorize import get_events, events_to_layer

In [6]:
da_drought = get_events(da, threshold=-1.0, extreme="below", outname="drought")
da_drought

<xarray.DataArray 'drought' (time: 660, location: 1118)> Size: 738kB
array([[False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * time      (time) datetime64[ns] 5kB 1960-01-31 1960-02-29 ... 2014-12-31
  * location  (location) object 9kB MultiIndex
  * lat       (location) float64 9kB 17.25 17.25 17.25 ... 54.75 54.75 54.75
  * lon       (location) float64 9kB 72.0 73.5 75.0 76.5 ... 132.0 133.5 135.0
Attributes:
    threshold:  -1.0
    extreme:    below

In [7]:
# da_pluvial = get_events(da, threshold=1.0, extreme="above", outname="pluvial")
# ds_layer = events_to_layer((da_drought, da_pluvial))
# ds_layer

# Calculate eca

In [17]:
import dominosee.eca
reload(dominosee.eca)
from dominosee.eca import get_eca_window, eca_dataset

In [18]:
da_drought_precwindow, da_drought_trigwindow = get_eca_window(da_drought)
da_drought_precwindow

<xarray.DataArray 'drought' (location: 1118, time: 660)> Size: 738kB
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True, False],
       ...,
       [ True,  True,  True, ...,  True,  True, False],
       [False, False, False, ...,  True,  True, False],
       [False, False, False, ...,  True,  True, False]])
Coordinates:
  * time      (time) datetime64[ns] 5kB 1960-01-31 1960-02-29 ... 2014-12-31
  * location  (location) object 9kB MultiIndex
  * lat       (location) float64 9kB 17.25 17.25 17.25 ... 54.75 54.75 54.75
  * lon       (location) float64 9kB 72.0 73.5 75.0 76.5 ... 132.0 133.5 135.0
Attributes:
    long_name:    Precursor Window
    units:        boolean
    description:  Window for precursor event identification
    eca_params:   {"delt": 2, "sym": true, "tau": 0}

In [19]:
# da_prec, da_trig = eca_dataset(da_drought, da_drought, da_drought_precwindow, da_drought_trigwindow)
# da_prec

In [20]:
# da_prec, da_trig = eca_dataset(da_drought, da_drought, da_drought_precwindow, da_drought_trigwindow, parallel=False)
# da_prec

In [21]:
import dask.config
dask.config.set(scheduler='threads')

In [22]:
da_prec, da_trig = eca_dataset(da_drought.chunk({"location": 13}), da_drought.chunk({"location": 13}), 
                               da_drought_precwindow.chunk({"location": 13}), da_drought_trigwindow.chunk({"location": 13}))
da_prec = da_prec.compute()
da_trig = da_trig.compute()
da_prec

<xarray.DataArray 'prec_evt' (drought_locationA: 1118, drought_locationB: 1118)> Size: 2MB
array([[ 70,  58,  49, ...,  51,  50,  45],
       [ 56,  82,  58, ...,  53,  55,  48],
       [ 52,  60,  69, ...,  51,  52,  48],
       ...,
       [ 44,  46,  42, ..., 115,  97,  91],
       [ 42,  48,  46, ..., 103, 108, 100],
       [ 36,  43,  40, ...,  94,  95, 110]], dtype=uint16)
Coordinates:
  * drought_locationA  (drought_locationA) object 9kB MultiIndex
  * lat_locA           (drought_locationA) float64 9kB 17.25 17.25 ... 54.75
  * lon_locA           (drought_locationA) float64 9kB 72.0 73.5 ... 133.5 135.0
  * drought_locationB  (drought_locationB) object 9kB MultiIndex
  * lat_locB           (drought_locationB) float64 9kB 17.25 17.25 ... 54.75
  * lon_locB           (drought_locationB) float64 9kB 72.0 73.5 ... 133.5 135.0
Attributes:
    long_name:    Precursor Events
    units:        count
    dtype:        uint16
    description:  Number of precursor events (from location A to location B) ...
    eca_params:   {"delt": 2, "sym": true, "tau": 0}

# Examine ECA significance

In [24]:
reload(dominosee.eca)
from dominosee.eca import get_prec_significance

In [34]:
da_prec_sig = get_prec_significance(da_prec, da_drought, da_drought)

<xarray.DataArray 'prec_sig' (drought_locationA: 1118, drought_locationB: 1118)> Size: 5MB
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , ..., 0.999715  , 0.9999936 ,
        0.9978128 ],
       [0.        , 1.        , 1.        , ..., 0.99997944, 0.99999917,
        0.9999437 ],
       ...,
       [0.        , 0.9981674 , 0.99975085, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.99989635, 0.99999917, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.9937561 , 0.99938655, ..., 1.        , 1.        ,
        1.        ]], dtype=float32)
Coordinates:
  * drought_locationA  (drought_locationA) object 9kB MultiIndex
  * lat_locA           (drought_locationA) float64 9kB 17.25 17.25 ... 54.75
  * lon_locA           (drought_locationA) float64 9kB 72.0 73.5 ... 133.5 135.0
  * drought_locationB  (drought_locationB) object 9kB MultiIndex
  * lat_locB           (drought_locationB) float64 9kB 17.25 17.25 ... 54.75
  * lon_locB           (drought_locationB) float64 9kB 72.0 73.5 ... 133.5 135.0
Attributes:
    long_name:    Precursor Significance
    units:        
    description:  Significance of precursor rates (from location A to locatio...
    eca_params:   {"delt": 2, "sym": true, "tau": 0}